In [2]:
%pip install autocorrect
%pip install -q datasets==2.16.1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import datasets
assert datasets.__version__ == "2.16.1"

In [6]:
import os
import numpy as np
import torch
import random
SEED = 42
# Seed the same seed to all
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


seed_everything(SEED)

In [10]:
from datasets import load_dataset
DATA_PATH = "../../pll_data_detection/data"
train_dataset = (
    load_dataset(
        "json", data_files=f"{DATA_PATH}/train.json", split="train"
    )
)

Generating train split: 0 examples [00:00, ? examples/s]

In [11]:
train_dataset

Dataset({
    features: ['trailing_whitespace', 'tokens', 'document', 'labels', 'full_text'],
    num_rows: 6807
})

In [ ]:
# 誤字の修正
import os 
from autocorrect import Speller
spell = Speller(lang='en')

def correct_spell_full_text(example):
    corrected_text = spell(example['full_text'])
    return {"full_text": corrected_text}

def correct_spell_token(example):
    return {"tokens": [spell(token) for token in example["tokens"]]}

# 本当はfull_textの修正を行う必要はないが、やっておくと安心
train_dataset = train_dataset.map(correct_spell_full_text, num_proc=os.cpu_count())
train_dataset = train_dataset.map(correct_spell_token, num_proc=os.cpu_count())

Map (num_proc=8):   0%|          | 0/6807 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/6807 [00:00<?, ? examples/s]

In [ ]:
# train_dataset.save_to_disk("../../pll_data_detection/data/train_spell_corrected")

In [ ]:
train_dataset.to_json("../../pll_data_detection/data/train_spell_corrected.json")